## Install dependencies

In [1]:
!pip install trankit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.4/773.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 45.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidi

### Prepare the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Run the training loop

In [3]:
import trankit
import torch

# Check if CUDA is available, otherwise use CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# initialize a trainer for the task
trainer = trankit.TPipeline(
    training_config={
      'max_epoch': 5,
      'category': 'customized-ner',  # pipeline category
      'task': 'ner', # task name
      'save_dir': '/content/drive/My Drive/data/syllable/save_dir_filtered', # directory to save the trained model
      'train_bio_fpath': '/content/drive/My Drive/data/syllable/train_syllable.conll', # training data in BIO format
      'dev_bio_fpath': '/content/drive/My Drive/data/syllable/dev_syllable.conll' # training data in BIO format
    }
)

# start training
trainer.train()

Setting up training config...


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Skipped 0 over-length examples
Loaded 5027 examples
Skipped 0 over-length examples
Loaded 2000 examples


Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

Saving adapter weights to ... /content/drive/My Drive/data/syllable/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner.ner.mdl (9.99 MB)
------------------------------
Best dev F1 score: epoch 2, F1: 88.09
******************************
NER: Epoch: 3


Train 3: 315it [01:00,  5.25it/s]
dev 3: 100%|█████████████████████████████| 125/125 [00:13<00:00,  9.11it/s]


Saving adapter weights to ... /content/drive/My Drive/data/syllable/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner.ner.mdl (9.99 MB)
------------------------------
Best dev F1 score: epoch 3, F1: 90.36
******************************
NER: Epoch: 4


Train 4: 315it [01:00,  5.25it/s]
dev 4: 100%|█████████████████████████████| 125/125 [00:13<00:00,  9.22it/s]


Saving adapter weights to ... /content/drive/My Drive/data/syllable/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner.ner.mdl (9.99 MB)
------------------------------
Best dev F1 score: epoch 4, F1: 90.84
Training done!


In [4]:
from trankit.iterators.ner_iterators import NERDataset
test_set = NERDataset(
    config=trainer._config,
    bio_fpath='/content/drive/My Drive/data/syllable/test_syllable.conll',
    evaluate=True
)
test_set.numberize()
test_batch_num = len(test_set) // trainer._config.batch_size + (len(test_set) % trainer._config.batch_size != 0)
result = trainer._eval_ner(data_set=test_set, batch_num=test_batch_num,
                           name='test', epoch=-1)

Skipped 0 over-length examples
Loaded 3000 examples


test -1: 100%|███████████████████████████| 188/188 [00:20<00:00,  9.00it/s]


In [5]:
print(result)

{'p': 90.24492534809596, 'r': 91.68299957392416, 'f1': 90.95827873356723}


## Running the trained model on the `test` set

In [6]:
!wget -q http://nlp.uoregon.edu/download/trankit/vietnamese.zip

In [7]:
%%shell
unzip -o vietnamese.zip

Archive:  vietnamese.zip
 extracting: vietnamese.downloaded   
  inflating: vietnamese.ner.mdl      
  inflating: vietnamese.ner-vocab.json  
  inflating: vietnamese.tagger.mdl   
  inflating: vietnamese.tokenizer.mdl  
  inflating: vietnamese.vocabs.json  


In [30]:
import os

save_dir = '/content/drive/My Drive/save_dir_filtered/xlm-roberta-base/customized-ner/preds/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f"Created directory: {save_dir}")
else:
    print(f"Directory already exists: {save_dir}")


Created directory: /content/drive/My Drive/save_dir_filtered/xlm-roberta-base/customized-ner/preds/


In [31]:
# initialize a trainer for the task
trainer1 = trankit.TPipeline(
    training_config={
      'max_epoch': 5,
      'category': 'customized-ner',  # pipeline category
      'task': 'lemmatize', # task name
      'save_dir': '/content/drive/My Drive/save_dir_filtered', # directory to save the trained model
      'train_conllu_fpath': '/content/drive/My Drive/data/vi_vtb-ud-train.conllu', # training data in BIO format
      'dev_conllu_fpath': '/content/drive/My Drive/data/vi_vtb-ud-dev.conllu' # training data in BIO format
    }
)

# start training
trainer1.train()

Setting up training config...
Initialized lemmatizer trainer
Training dictionary-based lemmatizer
Dev F1 = 96.64
Training seq2seq-based lemmatizer


Train 1: 100%|███████████████████████████| 405/405 [00:11<00:00, 36.27it/s]


Ensembling dict with seq2seq model
Saving new best model to ... /content/drive/My Drive/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner_lemmatizer.pt
Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |    100.00 |    100.00 |    100.00 |    100.00
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |    100.00 |    100.00 |    100.00 |    100.00
AllTags    |    100.00 |    100.00 |    100.00 |    100.00
Lemmas     |     98.48 |     98.48 |     98.48 |     98.48
UAS        |    100.00 |    100.00 |    100.00 |    100.00
LAS        |    100.00 |    100.00 |    100.00 |    100.00
CLAS       |    100.00 |    100.00 |    100.00 |    100.00
MLAS       |    100.00 |    100.00 |    100.00 |    100.00
BLEX       |     98.72 |     98.72 |

Train 2: 100%|███████████████████████████| 405/405 [00:10<00:00, 38.45it/s]


Ensembling dict with seq2seq model
Saving new best model to ... /content/drive/My Drive/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner_lemmatizer.pt
Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |    100.00 |    100.00 |    100.00 |    100.00
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |    100.00 |    100.00 |    100.00 |    100.00
AllTags    |    100.00 |    100.00 |    100.00 |    100.00
Lemmas     |     98.88 |     98.88 |     98.88 |     98.88
UAS        |    100.00 |    100.00 |    100.00 |    100.00
LAS        |    100.00 |    100.00 |    100.00 |    100.00
CLAS       |    100.00 |    100.00 |    100.00 |    100.00
MLAS       |    100.00 |    100.00 |    100.00 |    100.00
BLEX       |     98.85 |     98.85 |

...

Ensembling dict with seq2seq model
Saving new best model to ... /content/drive/My Drive/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner_lemmatizer.pt
Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |    100.00 |    100.00 |    100.00 |    100.00
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |    100.00 |    100.00 |    100.00 |    100.00
AllTags    |    100.00 |    100.00 |    100.00 |    100.00
Lemmas     |     99.26 |     99.26 |     99.26 |     99.26
UAS        |    100.00 |    100.00 |    100.00 |    100.00
LAS        |    100.00 |    100.00 |    100.00 |    100.00
CLAS       |    100.00 |    100.00 |    100.00 |    100.00
MLAS       |    100.00 |    100.00 |    100.00 |    100.00
BLEX       |     98.95 |     98.95 |

Train 4: 100%|███████████████████████████| 405/405 [00:10<00:00, 37.75it/s]


Ensembling dict with seq2seq model
Saving new best model to ... /content/drive/My Drive/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner_lemmatizer.pt
Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |    100.00 |    100.00 |    100.00 |    100.00
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |    100.00 |    100.00 |    100.00 |    100.00
AllTags    |    100.00 |    100.00 |    100.00 |    100.00
Lemmas     |     99.27 |     99.27 |     99.27 |     99.27
UAS        |    100.00 |    100.00 |    100.00 |    100.00
LAS        |    100.00 |    100.00 |    100.00 |    100.00
CLAS       |    100.00 |    100.00 |    100.00 |    100.00
MLAS       |    100.00 |    100.00 |    100.00 |    100.00
BLEX       |     98.95 |     98.95 |

Train 5: 100%|███████████████████████████| 405/405 [00:09<00:00, 42.98it/s]


Ensembling dict with seq2seq model
Saving new best model to ... /content/drive/My Drive/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner_lemmatizer.pt
Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |    100.00 |    100.00 |    100.00 |    100.00
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |    100.00 |    100.00 |    100.00 |    100.00
AllTags    |    100.00 |    100.00 |    100.00 |    100.00
Lemmas     |     99.27 |     99.27 |     99.27 |     99.27
UAS        |    100.00 |    100.00 |    100.00 |    100.00
LAS        |    100.00 |    100.00 |    100.00 |    100.00
CLAS       |    100.00 |    100.00 |    100.00 |    100.00
MLAS       |    100.00 |    100.00 |    100.00 |    100.00
BLEX       |     98.96 |     98.96 |

In [17]:
!mkdir -p "/content/drive/My Drive/data/syllable/save_dir_filtered/customized-ner/"
!cp -a vietnamese.tagger.mdl "/content/drive/My Drive/data/syllable/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner.tagger.mdl"
!cp -a vietnamese.vocabs.json "/content/drive/My Drive/data/syllable/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner.vocabs.json"
!cp -a vietnamese.tokenizer.mdl "/content/drive/My Drive/data/syllable/save_dir_filtered/xlm-roberta-base/customized-ner/customized-ner.tokenizer.mdl"



In [32]:
trankit.verify_customized_pipeline(
    category='customized-ner', # pipeline category
    save_dir='/content/drive/My Drive/data/syllable/save_dir_filtered', # directory used for saving models in previous steps
    embedding_name='xlm-roberta-base'
)

Customized pipeline is ready to use!
It can be initialized as follows:
-----------------------------------
from trankit import Pipeline
p = Pipeline(lang='customized-ner', cache_dir='/content/drive/My Drive/data/syllable/save_dir_filtered')


## Using the pretrained model in a Pipeline

In [19]:
from trankit import Pipeline

In [33]:
p = Pipeline(lang='customized-ner', cache_dir='/content/drive/My Drive/data/syllable/save_dir_filtered')

Loading pretrained XLM-Roberta, this may take a while...
Loading tokenizer for customized-ner
Loading tagger for customized-ner
Loading lemmatizer for customized-ner
Loading NER tagger for customized-ner
Active language: customized-ner


In [36]:
p.ner('''75 máy bay chiến đấu được sản xuất tại Hatfield, Anh và bay tới Thụy Sĩ vào năm 1949.''', is_sent=True)

{'text': '75 máy bay chiến đấu được sản xuất tại Hatfield, Anh và bay tới Thụy Sĩ vào năm 1949.',
 'tokens': [{'id': 1, 'text': '75', 'span': (0, 2), 'ner': 'O'},
  {'id': 2, 'text': 'máy bay chiến đấu', 'span': (3, 20), 'ner': 'O'},
  {'id': 3, 'text': 'được', 'span': (21, 25), 'ner': 'O'},
  {'id': 4, 'text': 'sản xuất', 'span': (26, 34), 'ner': 'O'},
  {'id': 5, 'text': 'tại', 'span': (35, 38), 'ner': 'O'},
  {'id': 6, 'text': 'Hatfield', 'span': (39, 47), 'ner': 'S-LOCATION'},
  {'id': 7, 'text': ',', 'span': (47, 48), 'ner': 'O'},
  {'id': 8, 'text': 'Anh', 'span': (49, 52), 'ner': 'S-LOCATION'},
  {'id': 9, 'text': 'và', 'span': (53, 55), 'ner': 'O'},
  {'id': 10, 'text': 'bay', 'span': (56, 59), 'ner': 'O'},
  {'id': 11, 'text': 'tới', 'span': (60, 63), 'ner': 'O'},
  {'id': 12, 'text': 'Thụy Sĩ', 'span': (64, 71), 'ner': 'S-LOCATION'},
  {'id': 13, 'text': 'vào', 'span': (72, 75), 'ner': 'O'},
  {'id': 14, 'text': 'năm', 'span': (76, 79), 'ner': 'O'},
  {'id': 15, 'text': '1949